In [18]:
# Reading .properties file, returns a dictionary
def loadProperties(filepath, sep='=', com='#'):
    props = {}
    with open(filepath, "rt") as f:
        for line in f:
            l = line.strip()
            if l and not l.startswith(com):
                key_value = l.split(sep)
                key = key_value[0].strip()
                value = key_value[1].strip()
                try:
                    props[value].append(key)
                except:
                    props[value] = [key]
    return props

In [28]:
props = loadProperties('submitActionClass.properties')

# Thats how props looks now
print('6b : ', props['6b'])
print('1a1 : ', props['1a1'])
print('6a2 : ', props['6a2'])

6b :  ['taiwan']
1a1 :  ['exmouth', 'tantabiddi']
6a2 :  ['cebu', 'oslob', 'sumilon', 'moalboal', 'kawasan', 'bohol']


In [75]:
import csv

# Reading WEKA file, returning a dictionary
def loadWEKA(filename, limit=0):
    attributes = [] # Will store csv columns names here
    data = []       # Will store readed file here
    
    with open(filename, "r") as csv_file:
        reader = csv.reader(csv_file, delimiter=',', quotechar="'")
        
        line_num = 0
        dataBegan = False
        
        for row in reader:
            # Getting info from @ATTRIBUTE's
            if not dataBegan:
                if len(row) > 0:
                    # Extracting @ATTRIBUTE's
                    row_splitted = row[0].split()
                    if row_splitted[0] == "@ATTRIBUTE":
                        attributes.append(row_splitted[1])
                    
                    # If we found that @data started
                    if row[0] == "@data":
                        dataBegan = True
                        continue
            
            # Reading only payload of the file
            if dataBegan:
                # Appending row into data array
                data.append(row)
                line_num += 1
                
                # Limiting the number of rows to read
                if line_num > limit and limit > 0:
                    break
        
    return (data, attributes)

In [83]:
(weka, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)

print("Thats how weka file looks like now. Array of Arrays:\n")
print(weka[:10])
print('\n\n')

print("Thats the column names loadWEKA() extracted:\n")
print(attr)

Thats how weka file looks like now. Array of Arrays:

[['1892013124614', '', '', '', '', 'East Contoy Island Mexico', '2c'], ['1892013125327', '', '', '', '', 'East Contoy Island Mexico', '2c'], ['189201375730', '', '', '', '', 'Red Sea  Marsa Alam   Dive Elphinstone Reef  North plateau', 'RedSea'], ['18a3e0a1-60fb-41e6-91a3-e887b5110d16', '', '', '', '', 'Red Sea', 'RedSea-ShibHabil'], ['18aeea70-955b-4cc4-ad19-f195ffa458d0', '', '', '', '', 'Talisayan Misamis Oriental Philippines', '6a'], ['18af726f-e98d-4ebb-a8ad-a91df6d9b78f', '', '', '', '', 'Black Rock Atoll Tubbataha Reef Palawan', '6a4'], ['18b6a564-4320-48bf-a61c-bc9613c547d8', '', '', '', '', 'Gulf of Tadjoura Djibouti Africa', 'Djibouti'], ['18cc82f0-03b8-493d-a2bc-3ee7ef659b0d', '', '', '', '', 'St Helena Island South Atlantic', '2h1'], ['18dd9c81-92d8-4ecd-b433-ee827a78f00b', '', '', '', '', 'Ningaloo Reef Western Australia', '1a'], ['1910200614191', '', '', '', '', 'Off Black Hills Utila Honduras', '2a']]



Thats the col